# 特征：
- 用户使用某类型优惠券数
- 用户领取优惠券数目
- 用户领取某种类型优惠券数
- 用户使用某种类型优惠券数目/领取该类型优惠券数目
- 用户使用某类型优惠券数/总体使用优惠券数
- 用户使用优惠券的最大距离
- 用户使用优惠券的最小距离
- 用户一天内领取优惠券的数目
- 用户消费的平均时间
- 用户一天领取多少种优惠券
- 用户使用某商家某种类型优惠券数目
- 用户使用某商家某类型的优惠券/用户在某商家所有使用的优惠券数目
- 用户使用某商家某类型优惠券/用户领取某商家某优惠券总数
- 用户使用某商家某类型优惠券/用户领取某商家优惠券总数
- 用户使用某商家某类型优惠券/领取某商家某优惠券总数
- 用户使用商家的优惠券/商家使用优惠券数
- 满减or折扣
- 某类优惠券发放次数
- 某种优惠券使用次数
- 某种优惠券使用/某种优惠券领取

In [1]:
import pandas as pd
import numpy as np

off_train = pd.read_csv('E:/tianchi/ccf_offline_stage1_train.csv')
off_test = pd.read_csv('E:/tianchi/ccf_offline_stage1_test_revised.csv')
off_train = off_train.dropna(subset = ['Coupon_id'])

In [2]:
off_train.Distance = off_train.Distance.fillna(1)
off_test.Distance = off_test.Distance.fillna(1)

# 用户使用某类型优惠券数

In [3]:
a = off_train[['User_id', 'Discount_rate', 'Date']].groupby(['User_id', 'Discount_rate'], as_index=False).count()
a = a.rename(columns={'Date' : 'User_Consume_type'})

off_train = pd.merge(off_train, a, on=['User_id', 'Discount_rate'], how='left')
off_test = pd.merge(off_test, a, on=['User_id', 'Discount_rate'], how='left')

off_test = off_test.fillna(value = {'User_Consume_type' : 0})

# 用户领取优惠券数目

In [4]:
a = off_train[['User_id', 'Date_received']].groupby('User_id', as_index=False).count()
a = a.rename(columns={'Date_received' : 'User_Received'})

off_train = pd.merge(off_train, a, on='User_id', how='left')
off_test = pd.merge(off_test, a, on='User_id', how='left')

off_test = off_test.fillna(value = {'User_Received' : 0})

# 用户领取某种类型优惠券数

In [5]:
a = off_train[['User_id', 'Discount_rate', 'Date_received']].groupby(['User_id', 'Discount_rate'], as_index=False).count()
a = a.rename(columns={'Date_received' : 'User_Received_type'})

off_train = pd.merge(off_train, a, on=['User_id', 'Discount_rate'], how='left')
off_test = pd.merge(off_test, a, on=['User_id', 'Discount_rate'], how='left')

off_test = off_test.fillna(value = {'User_Received_type' : 0})

# 用户使用某种类型优惠券数目/领取该类型优惠券数目

In [6]:
a = off_train[['User_id', 'Discount_rate', 'Date']].groupby(['User_id', 'Discount_rate'], as_index=False).count() 
b = off_train[['User_id', 'Discount_rate', 'Date_received']].groupby(['User_id', 'Discount_rate'], as_index=False).count()
c = pd.merge(a, b, on = ['User_id', 'Discount_rate'], how = 'left')
def chufa(z):
    x = z.Date / z.Date_received
    return x
c['User_Consume_type__User_Received_type'] = c.apply(chufa, axis=1)
c = c.drop(columns=['Date', 'Date_received'])

off_train = pd.merge(off_train, c, on=['User_id', 'Discount_rate'], how='left')
off_test = pd.merge(off_test, c, on=['User_id', 'Discount_rate'], how='left')

off_test = off_test.fillna(value = {'User_Consume_type__User_Received_type' : 0})

# 用户使用某类型优惠券数/总体使用优惠券数

In [7]:
a = off_train[['User_id', 'Discount_rate', 'Date']].groupby(['User_id', 'Discount_rate']).count() / off_train[['User_id', 'Date']].groupby('User_id').count()
a = a.reset_index()
a = a.dropna(subset=['Date'])
a = a.rename(columns={'Date' : 'User_Consume_type/User_Consume'})

off_train = pd.merge(off_train, a, on=['User_id', 'Discount_rate'], how='left')
off_test = pd.merge(off_test, a, on=['User_id', 'Discount_rate'], how='left')

off_test = off_test.fillna(value = {'User_Consume_type/User_Consume' : 0})

# 用户使用优惠券的最大距离

In [8]:
a = off_train[['User_id', 'Distance', 'Date']].groupby(['User_id', 'Date'], as_index=False).max()
a = a.rename(columns={'Distance' : 'User_Consume_Distance_max'})
a = a.drop_duplicates(subset=['User_id'])

off_train = pd.merge(off_train, a[['User_id', 'User_Consume_Distance_max']], on='User_id', how='left')
off_test = pd.merge(off_test, a[['User_id', 'User_Consume_Distance_max']], on='User_id', how='left')

off_test = off_test.fillna(value = {'User_Consume_Distance_max' : 10})

# 用户使用优惠券的最小距离

In [9]:
a = off_train[['User_id', 'Distance', 'Date']].groupby(['User_id', 'Date'], as_index=False).min()
a = a.rename(columns={'Distance' : 'User_Consume_Distance_min'})
a = a.drop_duplicates(subset=['User_id'])

off_train = pd.merge(off_train, a[['User_id', 'User_Consume_Distance_min']], on='User_id', how='left')
off_test = pd.merge(off_test, a[['User_id', 'User_Consume_Distance_min']], on='User_id', how='left')

off_test = off_test.fillna(value = {'User_Consume_Distance_min' : 0})

# 用户一天内领取优惠券的数目

In [10]:
a = off_train[['User_id', 'Date_received', 'Coupon_id']].groupby(['User_id', 'Date_received'], as_index=False).count()
a = a.rename(columns={'Coupon_id' : 'User_Received_oneday'})
off_train = pd.merge(off_train, a, on=['User_id', 'Date_received'], how='left')

a = off_test[['User_id', 'Date_received', 'Coupon_id']].groupby(['User_id', 'Date_received'], as_index=False).count()
a = a.rename(columns={'Coupon_id' : 'User_Received_oneday'})
off_test = pd.merge(off_test, a, on=['User_id', 'Date_received'], how='left')

# 用户消费的平均时间

In [11]:
import datetime

def days(z):
    if str(z['Date_received']) != 'nan' and str(z['Date']) != 'nan':
        days = (datetime.datetime.strptime(str(int(z['Date'])), "%Y%m%d") - datetime.datetime.strptime(str(int(z['Date_received'])), "%Y%m%d"))
        return days.days
    
off_train['days'] = off_train.apply(days, axis = 1)

a = off_train[['User_id', 'days']].groupby('User_id', as_index=False).mean()
a = a.dropna(subset=['days'])
a = a.rename(columns={'days' : 'User_Consume_day_mean'})

off_train = pd.merge(off_train, a, on='User_id', how='left')
off_test = pd.merge(off_test, a, on='User_id', how='left')

off_test = off_test.fillna(value = {'User_Consume_day_mean' : 0})

off_train = off_train.drop(columns=['days'])

# 用户一天领取多少种优惠券

In [12]:
a = off_train[['User_id', 'Date_received', 'Discount_rate']].groupby(['User_id', 'Date_received'], as_index=False).count()
a = a.rename(columns={'Discount_rate' : 'User_Discount_oneday_type'})
off_train = pd.merge(off_train, a, on=['User_id', 'Date_received'], how='left')

a = off_test[['User_id', 'Date_received', 'Discount_rate']].groupby(['User_id', 'Date_received'], as_index=False).count()
a = a.rename(columns={'Discount_rate' : 'User_Discount_oneday_type'})
off_test = pd.merge(off_test, a, on=['User_id', 'Date_received'], how='left')

# 用户使用某商家某种类型优惠券数目

In [13]:
a = off_train[['User_id', 'Merchant_id', 'Discount_rate', 'Date']].groupby(['User_id', 'Merchant_id', 'Discount_rate'], as_index=False).count()
a = a.rename(columns={'Date': 'User_Merchant_Consume_Discount_type'})

off_train = pd.merge(off_train, a, on=['User_id', 'Merchant_id', 'Discount_rate'], how='left')
off_test = pd.merge(off_test, a, on=['User_id', 'Merchant_id', 'Discount_rate'], how='left')

off_test = off_test.fillna(value = {'User_Merchant_Consume_type' : 0})

# 用户使用某商家某类型的优惠券/用户在某商家所有使用的优惠券数目

In [14]:
a = off_train[['User_id', 'Merchant_id', 'Discount_rate', 'Date']].groupby(['User_id', 'Merchant_id', 'Discount_rate'], as_index=False).count() 
b = off_train[['User_id', 'Merchant_id', 'Date']].groupby(['User_id', 'Merchant_id'], as_index=False).count()
b = b.rename(columns={'Date' : 'Consume'})
c = pd.merge(a, b, on = ['User_id', 'Merchant_id'], how = 'left')
def chufa(z):
    if z.Date != 0 and z.Consume != 0:
        x = z.Date / z.Consume
        return x
    else:
        return np.NaN
c['User_Merchant_Consume_Discount_type__User_Merchant_Consume'] = c.apply(chufa, axis=1)
c = c.drop(columns=['Date', 'Consume'])

off_train = pd.merge(off_train, c, on=['User_id', 'Merchant_id', 'Discount_rate'], how='left')
off_test = pd.merge(off_test, c, on=['User_id', 'Merchant_id', 'Discount_rate'], how='left')

off_train = off_train.fillna(value = {'User_Merchant_Consume_type__User_Merchant_Consume' : 0})
off_test = off_test.fillna(value = {'User_Merchant_Consume_type__User_Merchant_Consume' : 0})

# 用户使用某商家某类型优惠券/用户领取某商家某优惠券总数

In [15]:
a = off_train[['User_id', 'Merchant_id', 'Discount_rate', 'Date']].groupby(['User_id', 'Merchant_id', 'Discount_rate'], as_index=False).count()
b = off_train[['User_id', 'Merchant_id', 'Discount_rate', 'Date_received']].groupby(['User_id', 'Merchant_id', 'Discount_rate'], as_index=False).count()

c = pd.merge(a, b, on = ['User_id', 'Merchant_id', 'Discount_rate'], how = 'left')
def chufa(z):
    x = z.Date / z.Date_received
    return x
c['User_Merchant_Consume_type__User_Merchant_Received_type'] = c.apply(chufa, axis=1)
c = c.drop(columns=['Date', 'Date_received'])

off_train = pd.merge(off_train, c, on=['User_id', 'Merchant_id', 'Discount_rate'], how='left')
off_test = pd.merge(off_test, c, on=['User_id', 'Merchant_id', 'Discount_rate'], how='left')

off_test = off_test.fillna(value = {'User_Merchant_Consume_type__User_Merchant_Received_type' : 0})

# 用户使用某商家某类型优惠券/用户领取某商家优惠券总数

In [16]:
a = off_train[['User_id', 'Merchant_id', 'Discount_rate', 'Date']].groupby(['User_id', 'Merchant_id', 'Discount_rate'], as_index=False).count()
b = off_train[['User_id', 'Merchant_id', 'Date_received']].groupby(['User_id', 'Merchant_id'], as_index=False).count()

c = pd.merge(a, b, on = ['User_id', 'Merchant_id'], how = 'left')
def chufa(z):
    x = z.Date / z.Date_received
    return x
c['User_Merchant_Consume_type__User_Merchant_Received'] = c.apply(chufa, axis=1)
c = c.drop(columns=['Date', 'Date_received'])

off_train = pd.merge(off_train, c, on=['User_id', 'Merchant_id', 'Discount_rate'], how='left')
off_test = pd.merge(off_test, c, on=['User_id', 'Merchant_id', 'Discount_rate'], how='left')

off_test = off_test.fillna(value = {'User_Merchant_Consume_type__User_Merchant_Received' : 0})

# 用户使用某商家某类型优惠券/领取某商家某优惠券总数

In [17]:
a = off_train[['User_id', 'Merchant_id', 'Discount_rate', 'Date']].groupby(['User_id', 'Merchant_id', 'Discount_rate'], as_index=False).count()
b = off_train[['Merchant_id', 'Discount_rate', 'Date_received']].groupby(['Merchant_id', 'Discount_rate'], as_index=False).count()

c = pd.merge(a, b, on = ['Merchant_id', 'Discount_rate'], how = 'left')
def chufa(z):
    x = z.Date / z.Date_received
    return x
c['User_Merchant_Consume_type__Merchant_Received_type'] = c.apply(chufa, axis=1)
c = c.drop(columns=['Date', 'Date_received'])

off_train = pd.merge(off_train, c, on=['User_id', 'Merchant_id', 'Discount_rate'], how='left')
off_test = pd.merge(off_test, c, on=['User_id', 'Merchant_id', 'Discount_rate'], how='left')

off_test = off_test.fillna(value = {'User_Merchant_Consume_type__Merchant_Received_type' : 0})

# 用户使用商家的优惠券/商家使用优惠券数

In [18]:
a = off_train[['User_id', 'Merchant_id', 'Date']].groupby(['User_id', 'Merchant_id']).count() / off_train[['Merchant_id', 'Date']].groupby('Merchant_id').count()
a = a.reset_index()
a = a.rename(columns={'Date' : 'User_Merchant_Consume__Merchant_Consume'})

off_train = pd.merge(off_train, a, on=['User_id', 'Merchant_id'], how='left')
off_test = pd.merge(off_test, a, on=['User_id', 'Merchant_id'], how='left')

off_test = off_test.fillna(value = {'User_Merchant_Consume__Merchant_Consume' : 0})

# 满减or折扣

In [ ]:
import re
regex = re.compile('^\d+:\d+$')

def discount(z):
    if re.match(regex, str(z['Discount_rate'])):
        x = 1
        return x 
    else:
        x = 0
        return x

off_train['reduction_or_discount'] = off_train.apply(discount, axis = 1)
off_test['reduction_or_discount'] = off_test.apply(discount, axis = 1)

# 某类优惠券发放次数

In [ ]:
a = off_train.Discount_rate.value_counts()
a = a.reset_index()
a = a.rename(columns={'index' : 'Discount_rate', 'Discount_rate' : 'Received_type'})
from sklearn.preprocessing import minmax_scale 
b = minmax_scale(a.Received_type)
a.Received_type = b
off_train = pd.merge(off_train, a, on='Discount_rate', how='left')

a = off_test.Discount_rate.value_counts()
a = a.reset_index()
a = a.rename(columns={'index' : 'Discount_rate', 'Discount_rate' : 'Received_type'})
b = minmax_scale(a.Received_type)
a.Received_type = b
off_test = pd.merge(off_test, a, on='Discount_rate', how='left')

# 某种优惠券使用次数

In [ ]:
a = off_train[['Discount_rate', 'Date']].groupby('Discount_rate', as_index=False).count()
a = a.rename(columns={'Date' : 'Consume_type'})

off_train = pd.merge(off_train, a, on='Discount_rate', how='left')
off_test = pd.merge(off_test, a, on='Discount_rate', how='left')

off_test = off_test.fillna(value = {'Consume_type' : 0})

# 某种优惠券使用/某种优惠券领取

In [ ]:
a = off_train[['Discount_rate', 'Date']].groupby('Discount_rate', as_index=False).count() 
b = off_train[['Discount_rate', 'Date_received']].groupby('Discount_rate', as_index=False).count()
c = pd.merge(a, b, on='Discount_rate', how='left')
def chufa(z):
    x = z.Date / z.Date_received
    return x
c['Consume_type__Received_type'] = c.apply(chufa, axis=1)
c = c.drop(columns=['Date', 'Date_received'])

off_train = pd.merge(off_train, c, on='Discount_rate', how='left')
off_test = pd.merge(off_test, c, on='Discount_rate', how='left')

off_test = off_test.fillna(value = {'Consume_type__Received_type' : 0})